In [2]:
# Importando Bibliotecas
import os
import torch
import matplotlib.pyplot as plt
from torchvision.datasets import CIFAR10

In [3]:
DATA_DIR = './'
CATEGORIES = ['Avião', 'Automóvel','Passáro','Gato','Cervo','Cachorro', 'Sapo', 'Cavalo', 'Navio', 'Caminhão']

In [10]:
# Dados 
cifar10_train = CIFAR10(DATA_DIR, train=True, download=True)
cifar10_test = CIFAR10(DATA_DIR, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
(len(cifar10_train), len(cifar10_test))

(50000, 10000)

In [23]:
import torchvision.transforms as tt
prep_transform = tt.Compose([tt.ToTensor(), tt.Normalize(
    (0.4914, 0.4822, 0.4465),
    (0.2470, 0.2435, 0.2616)
)])
tensor_train = CIFAR10(DATA_DIR, train=True, download=False, transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=False, transform=prep_transform)

In [24]:
imgs = torch.stack([img_t for img_t, _ in tensor_train], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [21]:
imgs.view(3, -1).mean(dim=1) # Média das Cores

tensor([0.4914, 0.4822, 0.4465])

In [22]:
imgs.view(3, -1).std(dim=1) # Desvio Padrão das Cores

tensor([0.2470, 0.2435, 0.2616])

In [25]:
from torch.utils.data import DataLoader
batch_size = 64

train_loader = DataLoader(tensor_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tensor_test, batch_size=batch_size, shuffle=True)

In [27]:
# Testando o MLP para classificar imagem

from torch import nn

class MLPClassifier(nn.Module) :
    def __init__(self) :
        super().__init__()

        self.flatten = nn.Flatten()

        self.layers = nn.Sequential(
            nn.Linear(32 * 32 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x) :
        v = self.flatten(x)
        return self.layers(v)

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Rodando na {device}')

Rodando na cpu


In [28]:
model = MLPClassifier().to(device)

In [29]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
lossfunc = nn.CrossEntropyLoss() 

In [30]:
def train(model, dataloader, lossfunc, optimizer) :
    model.train()
    cumloss = 0.0
    for imgs, labels in dataloader :
        imgs, labels = imgs.to(device), labels.to(device)
        pred = model(imgs)
        loss = lossfunc(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return cumloss / len(dataloader)

def validate(model, dataloader, loss_func) : 
    model.eval()
    cumloss = 0.0
    with torch.no_grad() : 
        for imgs, labels in dataloader : 
            imgs, labels = imgs.to(device), labels.to(device)
            pred = model(imgs)
            loss = loss_func(pred, labels)
            cumloss += loss.item()
        return cumloss / len(dataloader)

In [31]:
def plot_losses(losses):
    fig = plt.figure(figsize=(13, 5))
    ax = fig.gca()
    for loss_name, loss_values in losses.items():  
        ax.plot(loss_values, label=loss_name)
    ax.legend(fontsize="16")
    ax.set_xlabel("Iteração", fontsize="16")
    ax.set_ylabel("Perda", fontsize="16")
    ax.set_title("Perdas vs Iterações", fontsize="16")

In [ ]:
epochs = 31
train_losses = []
test_losses = []
for t in range(epochs) :
    train_loss = train(model, train_loader, lossfunc, optimizer)
    train_losses.append(train_loss)
    test_loss = validate(model, test_loader, lossfunc)
    test_losses.append(test_loss)
    if(t % 10 == 0) :
        print(f"[Época][{t}] - Erro de Treinamento : {train_loss} - Erro de Teste : {test_loss}")
    
    